In [14]:
import numpy as np
import torch
import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import copy

In [29]:
df = pd.read_csv("retail_sales_dataset.csv")

x = torch.tensor(df["Age"].values, dtype=torch.float32)
y = torch.tensor(df["Total Amount"].values, dtype=torch.float32)

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, shuffle=True)

In [40]:
model = nn.Sequential(
    nn.Linear(10, 24),
    nn.ReLU(),
    nn.Linear(24, 12),
    nn.ReLU(),
    nn.Linear(12, 6),
    nn.ReLU(),
    nn.Linear(6, 1)
)

In [32]:
loss_func = nn.MSELoss()
opt = optim.Adam(model.parameters(), lr = 0.001)

In [41]:
# hyperparameters
epochs = 10
batch_size = 10
batch_start = torch.arange(0, len(x_train), batch_size)

# store analysis
best_mse = 999999
best_weights = None
history = []

In [42]:
# training loop
for epoch in range(epochs):
    
    model.train()

    with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
        bar.set_description(f"Epoch {epoch}")

        for i in bar:
            
            # take batch
            x_batch = x_train[i:i+batch_size]
            y_batch = y_train[i:i+batch_size]

            # forward pass
            y_pred = model(x_batch)
            loss = loss_func(y_pred, y_batch)

            # backward pass
            opt.zero_grad()
            loss.backward()

            # update weights
            opt.step()

            bar.set_postfix(mse=float(loss))
    
    model.eval()
    y_pred = model(x_test)
    mse = float(loss_func(y_pred, y_test))
    history.append(mse)

    if mse < best_mse:
        best_mse = mse
        best_weights = copy.deepcopy(model.state_dict())

c:\Users\paoma\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x200 and 10x24)